## Coroutine
*비동기 프로그래밍을 여행하는 히치하이커를 위한 안내서*

pep342 coroutine via enhanced generators

- generator를 좀 더 개선해서 coroutine 개념으로 동작할 수 있는 pep를 제안했습니다.
- python2.5부터 채택되었습니다(2005년) https://www.python.org/dev/peps/pep-0342/

In [59]:
# 우리가 잘 아는 generator
def well_known_generator():
    yield 100
    yield 200
    yield 300

for x in well_known_generator():
    print(x)

# as-iterator-protocol
g = well_known_generator()
next(g), next(g), next(g)

100
200
300


(100, 200, 300)

지금까지는 generator가 값을 만들어내는 역할로 동작했지만, <span class="mark">generator를 좀 더 개선하여 값을 받아내는 역할로도 동작할 수 있게 만들었습니다.</span>

*(개발자스럽게 더 풀어서 설명하자면) 예전엔 generator가 `__next__`를 구현한 iterable한 객체였다면, 향상된 generator는 `send()`가 추가로 구현되어 있습니다.*

generator를 개선해서 양방향으로 데이터를 주고 받을 수 있는 채널을 구현하면서, generator는 coroutine으로도 동작가능합니다.

- 생성자를 generator라고 한다면, 소비자는 coroutine입니다. *coroutine은 꼭 그런 개념은 아닌데 generator와 섞이지 않게 이렇게 표현하는 듯 합니다.*
- 데이터를 받고자 하는 시점에서 흐름을 중지할 수 있고,
- 신호(데이터)를 보냄으로써 실행을 재게할 수 있는 특성을 갖습니다.
- 이렇게 외부와 데이터를 주고 받을 수 있으면서, 실행시점을 제어할 수 있는 작업단위를 일반적인 의미의 coroutine이라고 합니다.

In [52]:
# 이런모양입니다.
def well_known_generator():
    yield 100
    yield 200
    yield 300

# __next__
g = well_known_generator()
print(next(g), next(g), next(g))
    
# send()
g = well_known_generator()
print(next(g), g.send(None), g.send(None))

100 200 300
100 200 300


In [4]:
def well_known_generator():
    print('started')
    received = yield 100
    print(received)
    received = yield 200
    print(received)
    received = yield 300
    print(received)
    print('end')
    
g = well_known_generator()
print(next(g), g.send(None), g.send(None))

# generator와 동일하게 stopiteration으로 끝납니다.
g.send(None)

started
None
None
100 200 300
None
end


StopIteration: 

In [6]:
# yield는 assign되는 expression일때만 ()를 생략할 수 있습니다.
def well_known_generator():
    print('started')
    print((yield 100))
    print((yield 200))
    print((yield 300))
    print('end')
    
g = well_known_generator()
print(next(g), g.send(None), g.send(None))

# generator와 동일하게 stopiteration으로 끝납니다.
g.send(None)

started
None
None
100 200 300
None
end


StopIteration: 

In [43]:
# coroutine에서의 return
def coroutine():
    sum_ = 0
    while True:
        sum_ += yield sum_
    return sum_

coro = coroutine()
next(coro)

0

In [44]:
coro.send(10)
coro.send(20)
coro.send(30)

60

In [45]:
coro.send(None)

TypeError: unsupported operand type(s) for +=: 'int' and 'NoneType'

In [46]:
coro.send(10)

StopIteration: 

In [1]:
import asyncio
import aiohttp
import collections
import os
import time

from IPython.display import display, Image

POP20_CC = (
    ' CN IN US ID BR PK NG BD RU JP KR '
    'MX PH VN ET EG DE IR TR CD FR'
).split()
BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = 'downloads/'

async def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = await aiohttp.request('GET', url)
    image = await resp.read()
    path = os.path.join(DEST_DIR, f'{cc.lower()}.gif')
    with open(path, 'wb') as fp:
        fp.write(image)
    display(Image(url=path))
    resp.close()


if __name__ == '__main__':
    t0 = time.time()
    loop = asyncio.get_event_loop()
    coros = [asyncio.Task(get_flag(cc)) for cc in POP20_CC]
    counts = loop.run_until_complete(asyncio.wait(coros))
    loop.close()
    elapsed = time.time() - t0
    print('elapsed in {:.2f}s'.format(elapsed))


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1096d2f28>


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1096bc2e8>


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1096e4d68>


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x109706128>


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1096e4470>


...

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1096d2dd8>


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x109730208>


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1096d24e0>


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x1096d2a20>


elapsed in 2.69s
